In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12742,2021-10-22T17:00:00,ITA,19,Sicilia,38.115697,13.362357,264,44,308,6387,...,DECEDUTI: N. 1 IL 21/10/21 - N. 3 IL 20/10/21 ...,1.0,Sul numero complessivo dei tamponi molecolari ...,NaN,304992.0,0.0,3318469.0,3284361.0,ITG,ITG1
12743,2021-10-22T17:00:00,ITA,9,Toscana,43.769231,11.255889,213,22,235,4828,...,NaN,2.0,NaN,NaN,282135.0,4811.0,4716154.0,2159011.0,ITI,ITI1
12744,2021-10-22T17:00:00,ITA,10,Umbria,43.106758,12.388247,36,4,40,730,...,"Si fa presente che 1 dei ricoveri NON UTI, non...",1.0,NaN,NaN,64609.0,0.0,1169457.0,922699.0,ITI,ITI2
12745,2021-10-22T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,3,0,3,89,...,NaN,0.0,NaN,NaN,11419.0,846.0,111599.0,101916.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12742,2021-10-22T17:00:00,19,Sicilia,264,44,308,6387,6695,13,400,...,304992,6602830,2518072.0,1.0,304992.0,0.0,3318469.0,3284361.0,ITG,ITG1
12743,2021-10-22T17:00:00,9,Toscana,213,22,235,4828,5063,-7,277,...,286946,6875165,3013906.0,2.0,282135.0,4811.0,4716154.0,2159011.0,ITI,ITI1
12744,2021-10-22T17:00:00,10,Umbria,36,4,40,730,770,54,67,...,64609,2092156,451596.0,1.0,64609.0,0.0,1169457.0,922699.0,ITI,ITI2
12745,2021-10-22T17:00:00,2,Valle d'Aosta,3,0,3,89,92,-6,2,...,12265,213515,90794.0,0.0,11419.0,846.0,111599.0,101916.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-10-22', '2021-10-21')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-10-22T17:00:00,1,Piemonte,176,19,195,3445,3640,159,278,...,382962,7461754,2555002.0,3.0,360307.0,22655.0,3601763.0,3859991.0,ITC,ITC1
1,2021-10-22T17:00:00,2,Valle d'Aosta,3,0,3,89,92,-6,2,...,12265,213515,90794.0,0.0,11419.0,846.0,111599.0,101916.0,ITC,ITC2
2,2021-10-22T17:00:00,3,Lombardia,274,51,325,7850,8175,-141,381,...,890542,16555216,5723535.0,2.0,824063.0,66479.0,11128409.0,5426807.0,ITC,ITC4
3,2021-10-22T17:00:00,5,Veneto,165,26,191,8862,9053,88,381,...,476420,13152085,2162076.0,1.0,461394.0,15026.0,6983753.0,6168332.0,ITH,ITH3
4,2021-10-22T17:00:00,6,Friuli Venezia Giulia,50,7,57,1045,1102,53,138,...,115400,3021380,845568.0,1.0,99955.0,15445.0,2205178.0,816202.0,ITH,ITH4
5,2021-10-22T17:00:00,7,Liguria,55,12,67,798,865,23,86,...,114101,2519308,899680.0,2.0,114101.0,0.0,1691732.0,827576.0,ITC,ITC3
6,2021-10-22T17:00:00,8,Emilia-Romagna,287,30,317,10849,11166,-610,315,...,429090,9420318,2135251.0,1.0,428724.0,366.0,6066285.0,3354033.0,ITH,ITH5
7,2021-10-22T17:00:00,9,Toscana,213,22,235,4828,5063,-7,277,...,286946,6875165,3013906.0,2.0,282135.0,4811.0,4716154.0,2159011.0,ITI,ITI1
8,2021-10-22T17:00:00,10,Umbria,36,4,40,730,770,54,67,...,64609,2092156,451596.0,1.0,64609.0,0.0,1169457.0,922699.0,ITI,ITI2
9,2021-10-22T17:00:00,11,Marche,45,10,55,1955,2010,15,116,...,115324,1605260,931543.0,0.0,115324.0,0.0,1384379.0,220881.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-10-21T17:00:00,1,Piemonte,180,18,198,3283,3481,48,267,...,382684,7401219,2551510.0,1.0,360131.0,22553.0,3595628.0,3805591.0,ITC,ITC1
1,2021-10-21T17:00:00,2,Valle d'Aosta,3,0,3,95,98,-3,2,...,12263,211722,90655.0,0.0,11418.0,845.0,111512.0,100210.0,ITC,ITC2
2,2021-10-21T17:00:00,3,Lombardia,276,53,329,7987,8316,24,383,...,890161,16457563,5712759.0,1.0,823767.0,66394.0,11109192.0,5348371.0,ITC,ITC4
3,2021-10-21T17:00:00,5,Veneto,153,28,181,8784,8965,97,446,...,476039,13071143,2159059.0,1.0,460966.0,15073.0,6969809.0,6101334.0,ITH,ITH3
4,2021-10-21T17:00:00,6,Friuli Venezia Giulia,54,6,60,989,1049,91,149,...,115264,2999692,843716.0,0.0,99830.0,15434.0,2200486.0,799206.0,ITH,ITH4
5,2021-10-21T17:00:00,7,Liguria,50,11,61,781,842,33,94,...,114015,2505622,897823.0,0.0,114015.0,0.0,1688977.0,816645.0,ITC,ITC3
6,2021-10-21T17:00:00,8,Emilia-Romagna,283,31,314,11462,11776,-488,294,...,428776,9391735,2132717.0,3.0,428411.0,365.0,6053082.0,3338653.0,ITH,ITH5
7,2021-10-21T17:00:00,9,Toscana,210,22,232,4838,5070,-54,277,...,286669,6846133,3005419.0,2.0,281865.0,4804.0,4707436.0,2138697.0,ITI,ITI1
8,2021-10-21T17:00:00,10,Umbria,35,4,39,677,716,130,141,...,64542,2082165,450969.0,0.0,64542.0,0.0,1167463.0,914702.0,ITI,ITI2
9,2021-10-21T17:00:00,11,Marche,44,10,54,1941,1995,-2,100,...,115208,1602588,930228.0,0.0,115208.0,0.0,1382165.0,220423.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-10-22T17:00:00,1,Piemonte,176,19,195,3445,3640,159,278,...,7461754,2555002.0,3.0,360307.0,22655.0,3601763.0,3859991.0,ITC,ITC1,4.341375e+06
1,2021-10-22T17:00:00,2,Valle d'Aosta,3,0,3,89,92,-6,2,...,213515,90794.0,0.0,11419.0,846.0,111599.0,101916.0,ITC,ITC2,1.255010e+05
2,2021-10-22T17:00:00,3,Lombardia,274,51,325,7850,8175,-141,381,...,16555216,5723535.0,2.0,824063.0,66479.0,11128409.0,5426807.0,ITC,ITC4,1.010397e+07
3,2021-10-22T17:00:00,5,Veneto,165,26,191,8862,9053,88,381,...,13152085,2162076.0,1.0,461394.0,15026.0,6983753.0,6168332.0,ITH,ITH3,4.907704e+06
4,2021-10-22T17:00:00,6,Friuli Venezia Giulia,50,7,57,1045,1102,53,138,...,3021380,845568.0,1.0,99955.0,15445.0,2205178.0,816202.0,ITH,ITH4,1.211357e+06
5,2021-10-22T17:00:00,7,Liguria,55,12,67,798,865,23,86,...,2519308,899680.0,2.0,114101.0,0.0,1691732.0,827576.0,ITC,ITC3,1.543127e+06
6,2021-10-22T17:00:00,8,Emilia-Romagna,287,30,317,10849,11166,-610,315,...,9420318,2135251.0,1.0,428724.0,366.0,6066285.0,3354033.0,ITH,ITH5,4.467118e+06
7,2021-10-22T17:00:00,9,Toscana,213,22,235,4828,5063,-7,277,...,6875165,3013906.0,2.0,282135.0,4811.0,4716154.0,2159011.0,ITI,ITI1,3.722729e+06
8,2021-10-22T17:00:00,10,Umbria,36,4,40,730,770,54,67,...,2092156,451596.0,1.0,64609.0,0.0,1169457.0,922699.0,ITI,ITI2,8.802850e+05
9,2021-10-22T17:00:00,11,Marche,45,10,55,1955,2010,15,116,...,1605260,931543.0,0.0,115324.0,0.0,1384379.0,220881.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-10-22T17:00:00,1,Piemonte,176,19,195,3445,3640,159,278,...,3859991.0,ITC,ITC1,4.341375e+06,Piemonte,176,5824,19,628,99
1,2021-10-22T17:00:00,2,Valle d'Aosta,3,0,3,89,92,-6,2,...,101916.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,3,83,0,33,0
2,2021-10-22T17:00:00,3,Lombardia,274,51,325,7850,8175,-141,381,...,5426807.0,ITC,ITC4,1.010397e+07,Lombardia,274,6369,51,1530,0
3,2021-10-22T17:00:00,5,Veneto,165,26,191,8862,9053,88,381,...,6168332.0,ITH,ITH3,4.907704e+06,Veneto,165,6000,26,1000,0
4,2021-10-22T17:00:00,6,Friuli Venezia Giulia,50,7,57,1045,1102,53,138,...,816202.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,50,1277,7,175,0
5,2021-10-22T17:00:00,7,Liguria,55,12,67,798,865,23,86,...,827576.0,ITC,ITC3,1.543127e+06,Liguria,55,1700,12,217,12
6,2021-10-22T17:00:00,8,Emilia-Romagna,287,30,317,10849,11166,-610,315,...,3354033.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,287,7920,30,889,0
7,2021-10-22T17:00:00,9,Toscana,213,22,235,4828,5063,-7,277,...,2159011.0,ITI,ITI1,3.722729e+06,Toscana,213,5033,22,570,37
8,2021-10-22T17:00:00,10,Umbria,36,4,40,730,770,54,67,...,922699.0,ITI,ITI2,8.802850e+05,Umbria,36,662,4,84,43
9,2021-10-22T17:00:00,11,Marche,45,10,55,1955,2010,15,116,...,220881.0,ITI,ITI3,1.518400e+06,Marche,45,964,10,212,63


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-10-22T17:00:00,15,Campania,186,21,207,6334,6541,140,450,...,3.78,144,95,24350,3629.0,8,302,1.85,0.00778,7.99
1,2021-10-22T17:00:00,12,Lazio,312,49,361,8297,8658,53,408,...,5.20,55,-23,34877,7486.0,4,351,1.17,0.00696,6.66
2,2021-10-22T17:00:00,19,Sicilia,264,44,308,6387,6695,13,400,...,4.93,19,114,13733,4249.0,6,381,2.91,0.00805,6.14
3,2021-10-22T17:00:00,3,Lombardia,274,51,325,7850,8175,-141,381,...,3.33,-137,-2,97653,10776.0,4,518,0.39,0.00377,8.81
4,2021-10-22T17:00:00,5,Veneto,165,26,191,8862,9053,88,381,...,2.60,78,-65,80942,3017.0,1,292,0.47,0.00776,9.71
5,2021-10-22T17:00:00,8,Emilia-Romagna,287,30,317,10849,11166,-610,315,...,3.37,-613,21,28583,2534.0,2,922,1.10,0.00705,9.61
6,2021-10-22T17:00:00,1,Piemonte,176,19,195,3445,3640,159,278,...,3.03,162,11,60535,3492.0,2,117,0.46,0.00640,8.82
7,2021-10-22T17:00:00,9,Toscana,213,22,235,4828,5063,-7,277,...,3.86,-10,0,29032,8487.0,3,281,0.95,0.00744,7.71
8,2021-10-22T17:00:00,16,Puglia,127,16,143,2017,2160,97,210,...,3.32,105,43,20532,3052.0,3,110,1.02,0.00524,6.76
9,2021-10-22T17:00:00,18,Calabria,77,8,85,2739,2824,57,145,...,4.60,57,0,5200,3125.0,1,87,2.79,0.00753,4.48


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,P.A. Bolzano,6,94,100,6.00
1,Liguria,12,205,217,5.53
2,Lazio,49,894,943,5.20
3,Sicilia,44,849,893,4.93
4,Sardegna,10,194,204,4.90
5,Umbria,4,80,84,4.76
6,Marche,10,202,212,4.72
7,Calabria,8,166,174,4.60
8,Friuli Venezia Giulia,7,168,175,4.00
9,Toscana,22,548,570,3.86


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))